<h1>Quantum Simualtor</h1>

<h2>A Quantum computer simulator with support of upto 3 qbits</h2>

In [1]:
import numpy as np
import random

In [2]:
def get_ground_state(num_qubits):
    # returns vector of size 2**num_qubits with all zeroes except first element which is 1
    ground_state_vector = [0]*2**num_qubits
    ground_state_vector[0]=1 
    
    return ground_state_vector

In [15]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    
    # unitary gates --> 
    I = np.identity(2)
    H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)],[1/np.sqrt(2), -1/np.sqrt(2)]])
    X = np.array([[0, 1],[1, 0]])
    P0x0 = np.array([[1, 0],[0, 0]])
    P1x1 = np.array([[0, 0],[0, 1]])
    CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
    

    
    # Initilized H gate 
    if gate_unitary=="h":
        if 0 in target_qubits:
            gate = H
        else:
            gate = I
        if gate_unitary=="h":
            for i in range(1,total_qubits):
                if i in target_qubits:
                    gate = np.kron(gate,H)
                else:
                    gate = np.kron(gate,I)
                    
    elif gate_unitary == 'cx' and total_qubits==2:
        gate = CNOT
    
    elif gate_unitary == "cx" and total_qubits==3:
        if target_qubits == [0,1]:
            gate = np.kron(CNOT,I)
        elif target_qubits == [1,2]:
            gate = np.kron(I,CNOT)
        elif target_qubits == [1,3]:
            gate = np.kron(np.kron(P0x0, I), I) + np.kron(np.kron(P1x1, I), X)
            
    return gate


In [21]:
def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    
    steps= len(program)
    num_qbits = int(np.log2(len(initial_state)))
    current_state=initial_state
    for i in range(steps):
        current_state = np.dot(current_state,get_operator(num_qbits,program[i]["gate"],program[i]["target"]))
    print(current_state)
    return current_state


In [6]:
def measure_all(state_vector):
    
    num_vector = len(state_vector)
    
    if num_vector == 2:
        states = ['0','1']
    elif num_vector==4:
        states = ['00','01','10','11']
    elif num_vector == 8:
        states = ['000','001','010','011','100','101','110','111']
    else:
        print("invalid state vector: number of qbits might exceed 3")

    choosen = random.choices(states, weights=np.abs(state_vector)**2,  k=1)
    return choosen

In [7]:
def get_counts(state_vector, num_shots):

    num_vector = len(state_vector)

    if num_vector == 2:
        counts = {'0':0,'1':0}
    elif num_vector==4:
        counts = {'00':0,'01':0,'10':0,'11':0}
    elif num_vector == 8:
        counts = {'000':0,'001':0,'010':0,'011':0,'100':0,'101':0,'110':0,'111':0}
    else:
        print("invalid state vector: number of qbits might exceed 3")

    for i in range(num_shots):
        a = measure_all(state_vector)
        counts[a[0]] = counts[a[0]]+1
        
    
    
    
    return counts

In [18]:
my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]

In [22]:
# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(3)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)

[0.70710678 0.         0.         0.         0.         0.
 0.70710678 0.        ]
{'000': 488, '001': 0, '010': 0, '011': 0, '100': 0, '101': 0, '110': 512, '111': 0}


In [20]:
print(current_state)

NameError: name 'current_state' is not defined